<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# Session: Using a Database to Find Solutions for Business Problems 

#### <i>A Microsoft Presentation from the SQL Server team</i>

## Querying the Database

<p style="border-bottom: 1px solid lightgrey;"></p>


<h2><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/textbubble.png?raw=true">Review the Solutions Database Structure</h2>

You can build your queries by studying the ERD for this solution. We're using a very simple layout for this database, involving only three primary tables:

- **Problem**: The list of problems in the form of "We want..."
- **Solution**: Solutions composed of various components, processes and optionally services
- **AzureService**: The Azure Services (components) and optionally any services (consulting/staff aug) needed for the solution

There are also two secondary tables to hold information about the solutions and services: 

- **SolutionReferences**: Reference documents, tutorials, and cost estimates for the Solution
- **ServiceReferences**: Reference documents, tutorials, and code samples for the Services


And of course tertiary tables to handle the many-to-many joins:

- **ProblemToSolution**
- **SoltuionToService**

Here is the Entity Relationship Diagram (ERD) for the database:

<p><img style="float: left; margin: 0px 35px 35px 0px;" src="https://github.com/BuckWoody/presentations/blob/master/problemtosolution/graphics/erd.png?raw=true"></p>

You can also edit this simple design to include more elements, such as more detail on the problem, or any other specific information you want.

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Enter a Problem, A Solution, and Tie Them Together</b></p>

- Thing:
- The following code cell shows an example:


In [1]:
-- Thing

Commands completed successfully.

Total execution time: 00:00:00.042

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Add Additional Information for a Solution</b></p>

- Thing:
- The following code cell shows an example:

In [0]:
-- Thing

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Enter an Azure Service, and Tie it to a Solution</b></p>

- Thing:
- The following code cell shows an example:

In [0]:
-- Thing

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Add Additional Information for a Service</b></p>

- Thing:
- The following code cell shows an example:

In [0]:
--  Thing

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Query Data Using the Included Views</b></p>

- Thing:
- The following code cell shows an example:

In [0]:
-- Thing

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/checkbox.png?raw=true"><b>Activity: Query Various Elements Of the Database</b></p>

- Thing:
- The following code cell shows an example:

In [0]:
-- Thing

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/thinking.jpg?raw=true"><b>For Further Study</b></p>

<br>
<br>

- [If you need to learn more about T-SQL queries, there's a good series here](https://channel9.msdn.com/Series/Querying-SQL-Server-2012)

<p><img style="float: left; margin: 0px 15px 15px 0px;" src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/education1.png?raw=true"><b>Next</b>: Extending the Database, using a Query Interface</p>

You can change this database to whatever you like, and if you want to set up a front-end program to view the data, there's a Power BI file included in this directory you can use as an example to get started. Note that you'll need to change the connection information for the Power BI file to your database.